In [1]:
import numpy as np
import pyvista as pv
from tqdm import tqdm

from kale import Engine, save_movie, show_ui
from kale.algorithms import (
    contour_banded,
    subdivide_algorithm,
    scalars_operation_algorithm,
)

In [2]:
# Requires PyVista 0.38.1
pv.set_jupyter_backend("server")
pv.global_theme.trame.interactive_ratio = 2
pv.global_theme.trame.still_ratio = 2
pv.global_theme.image_scale = 2  # upscales the saved screenshots/video frames

In [3]:
# MESH_GEOMETRY_FILE_NAME = "nankai_very_very_dense.vtk"
MESH_GEOMETRY_FILE_NAME = "nankai_121728_clean.vtk"
# MESH_VALUES_FILE_NAME = "2023_03_03_16_15_24.hdf"
MESH_VALUES_FILE_NAME = "2023_03_04_00_02_03.hdf"
MESH_Z_SCALE = 0.01
TIME_STEP_INTERESTING = 55849
TIME_STEP_INTERESTING = 1000

LINE_WIDTH = 1
CONTOUR_LINE_COLOR = "black"
COLOR_MAPS = {
    "cumulative_slip": "CET_L19",  #'CET_C3',
    "geometric_moment": "CET_R3",
    "last_event_slip": "CET_L19",
    "loading_rate": "CET_C3",
}

# TODO: The same fontsizes seem to produce different sized fonts on the figure
# I'd like all fonts to be the same size and type
COLORBAR_FONT_SIZE = 34
AXES_FONT_SIZE = 26
TIMESTEP_FONT_SIZE = 12

SCALAR_BAR_OPTS = dict(
    # Position (coordinates are on the viewport)
    vertical=True,
    height=0.20,
    width=0.05,
    position_x=0.05,
    position_y=0.40,
    # Labels
    title_font_size=AXES_FONT_SIZE,
    label_font_size=AXES_FONT_SIZE,
    n_labels=3,
    italic=False,
    fmt="%.1f",
    font_family="arial",
    shadow=True,
)

## Load Data

Set up the driver of the data IO and time step management with the `kale.Engine` class

In [4]:
engine = Engine(MESH_GEOMETRY_FILE_NAME, MESH_VALUES_FILE_NAME, zscale=MESH_Z_SCALE)
engine.keys

['cumulative_slip', 'geometric_moment', 'last_event_slip', 'loading_rate']

In [5]:
# Set which array to be active
engine.mesh.active_scalars_name = "geometric_moment"
engine.time_step = TIME_STEP_INTERESTING

# Updated video style

In [6]:
SCALAR_BAR_OPTS = dict(
    # Position (coordinates are on the viewport)
    vertical=True,
    height=0.00,
    width=0.00,
    position_x=0.05,
    position_y=0.40,
    # Labels
    title_font_size=0,
    label_font_size=0,
    n_labels=0,
    italic=False,
    fmt="%.1f",
    font_family="arial",
    shadow=True,
)

var = "geometric_moment"

engine.mesh.active_scalars_name = "geometric_moment"

# Manipulate the quantity
operation = np.cbrt
MANIPULATED_ARRAY_NAME = f"cbrt({engine.mesh.active_scalars_name})"
manipulated = scalars_operation_algorithm(
    engine.algorithm, operation, output_scalars_name=MANIPULATED_ARRAY_NAME
)

# CONTOUR_LEVELS = np.linspace(-10e9, 10e9, 11)
CONTOUR_LEVELS = np.linspace(-600, 600, 11)
N_COLORS = len(CONTOUR_LEVELS) - 1
CLIM = [np.min(CONTOUR_LEVELS), np.max(CONTOUR_LEVELS)]

pl = pv.Plotter()
pl.add_mesh(engine.boundary_smoothed, color="k", line_width=LINE_WIDTH)

contour, edges = contour_banded(
    manipulated,
    CONTOUR_LEVELS,
    rng=CLIM,
    scalars=MANIPULATED_ARRAY_NAME,
)

pl.add_mesh(
    contour,
    # cmap=COLOR_MAPS[var],
    # cmap="PuOr_r",
    cmap="RdYlBu_r",    
    clim=CLIM,
    n_colors=N_COLORS,
    scalar_bar_args=dict(title=f"", **SCALAR_BAR_OPTS),
)
pl.add_mesh(edges.GetOutputPort(1), color=CONTOUR_LINE_COLOR, line_width=LINE_WIDTH, edge_color="k")
pl.add_mesh(engine.boundary, color=CONTOUR_LINE_COLOR, line_width=LINE_WIDTH, edge_color="k")

box = pv.Box(engine.mesh.bounds)
pl.add_mesh(box, color="lightgrey", culling="front")
pl.camera.zoom(1.2)
pl.image_scale = 2

# Make a video over time steps
step = 5000
pl.open_movie(f"{var}_nankai_movie_only.mp4", framerate=60, quality=5)
for tindex in tqdm(range(0, engine.max_time_step, step)):
    engine.time_step = tindex
    pl.add_text(f"i = {tindex:05d}", name="time-step-label", position="lower_edge")
    pl.write_frame()
pl.close()


100%|██████████| 10/10 [00:02<00:00,  4.59it/s]
